In [1]:
import numpy as np
from sklearn.svm import LinearSVC
import sys
sys.path.append('../general')
from pool import Pool
from classification_counterfactual_model import ClassificationCounterfactualModel
from metric import metric
from collections import Counter

In [2]:
pool = Pool('../data')
pool.log_features()
train_pool, test_pool = pool.train_test_split()
train_pools = train_pool.split_by_position()

In [3]:
NONE_POSITION = 10
POSITIONS = np.arange(NONE_POSITION)
models = [
    LinearSVC()
    for position in POSITIONS
]
model = ClassificationCounterfactualModel(models)

In [4]:
def get_sample_weight(pool):
    proportion = len(pool.targets[pool.targets != 0]) / len(pool.targets[pool.targets == 0])
    return [proportion if target == 0 else 100 for target in pool.targets]

In [5]:
model.fit(train_pools, [{
        'sample_weight': get_sample_weight(tmp_train_pool)
    } for tmp_train_pool in train_pools
])

In [6]:
prediction = model.predict(test_pool)
score = metric(prediction, test_pool.positions, test_pool.targets, test_pool.probas)

In [7]:
score

-0.052525255694655985

In [8]:
Counter(prediction)

Counter({7: 2, 10: 26998})